# Introduction

This tutorial is about the transformation packages `LocalCoordinateSystem` class which describes the orientation and position of a cartesian coordinate system towards another reference coordinate system. The reference coordinate systems origin is always at $(0, 0, 0)$ and its orientation is described by the basis: $e_x = (1, 0, 0)$, $e_y = (0, 1, 0)$, $e_z = (0, 0, 1)$. 

The packages required in this tutorial are:

In [1]:
%matplotlib widget

In [2]:
# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

# interactive plots
import ipywidgets as widgets
from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import numpy as np

import weldx.visualization as vs
import weldx.transformations as tf

# Construction

The constructor of the `LocalCoordinateSystem` class takes 2 parameters, the `orientation` and the `coordinates`. `orientation` is a 3x3 matrix. It can either be viewed as a rotation/reflection matrix or a set of normalized column vectors that represent the 3 basis vectors of the coordinate system. The matrix needs to be orthogonal, otherwise an exception is raised. `coordinates` is the position of the local coordinate systems origin inside the reference coordinate system. The default parameters are the identity matrix and the zero vector. Hence we get a system that is identical to the reference system if no parameter is passed to the constructor.

In [16]:
cs_ref = tf.LocalCoordinateSystem()

We create some coordinate systems and visualize them using the `visualization` package. The coordinate axes are colored as follows: 
- x = red
- y = green
- z = blue

In [15]:
# create a translated coordinate system
cs_01 = tf.LocalCoordinateSystem(coordinates=[2, 4, -1])

# create a rotated coordinate system using a rotation matrix as basis
rotation_matrix = tf.rotation_matrix_z(np.pi / 3)
cs_02 = tf.LocalCoordinateSystem(orientation=rotation_matrix, coordinates=[0, 0, 3])

# create 3d plot
fig = plt.figure()
ax = fig.gca(projection="3d")
fig.canvas.layout.height = "500px"
fig.canvas.layout.width = "500px"

vs.plot_coordinate_system(cs_ref, ax, color="r", label="reference system")
vs.plot_coordinate_system(cs_01, ax, color="g", label="system 1")
vs.plot_coordinate_system(cs_02, ax, color="b", label="system 2")
vs.set_axes_equal(ax)
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

> **HINT:** In the jupyter notebook version of this tutorial, you can rotate the plot by pressing the left mouse button and moving the mouse. This helps to understand how the different coordinate systems are positioned in the 3d space.

Apart from the class constructor, there are some factory functions implemented to create a coordinate system. The `construct_from_orientation` provides the same functionality as the class constructor. The `construct_from_xyz` takes 3 basis vectors instead of a matrix. `construct_from_xy_and_orientation`, `construct_from_xz_and_orientation` and `construct_from_yz_and_orientation` create a coordinate system with 2 basis vectors and a bool which speciefies if the coordinate system should have a positive or negative orientation. Here are some examples:

In [14]:
# coordinate system using 3 basis vectors
e_x = [1, 2, 0]
e_y = [-2, 1, 0]
e_z = [0, 0, 5]
cs_03 = tf.LocalCoordinateSystem.construct_from_xyz(e_x, e_y, e_z, coordinates=[1, 1, 0])

# create a negatively oriented coordinate system with 2 vectors
cs_04 = tf.LocalCoordinateSystem.construct_from_yz_and_orientation(
    e_y, e_z, positive_orientation=False, coordinates=[1, 1, 2]
)

# create 3d plot
fig = plt.figure()
ax = fig.gca(projection="3d")
fig.canvas.layout.height = "500px"
fig.canvas.layout.width = "500px"

vs.plot_coordinate_system(cs_ref, ax, color="r", label="reference system")
vs.plot_coordinate_system(cs_03, ax, color="g", label="system 3")
vs.plot_coordinate_system(cs_04, ax, color="b", label="system 4")
vs.set_axes_equal(ax)
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As you can see, the y and z axis of system 3 and 4 point into the same direction, since we used the same basis vectors. The automatically determined x axis of system 4 points into the opposite direction, since we wanted a system with negative orientation.

Another method to create a `LocalCoordinateSystem` is `construct_from_euler`. It utilizes the `scipy.spatial.transform.Rotation.from_euler` function to calculate a rotion matrix from euler sequences and uses it to describe the orientation of the coordinate system. The parameters `sequence`, `angles` and `degrees` of the `construct_from_euler` method are directly passed to the SciPy function. `sequence` expects a string that determines the rotation sequence around the coordinate axes. For example `"xyz"`. `angles` is  a scalar or list of the corresponding number of angles and `degrees` a `bool` that specifies if the angles are provided in degrees (`degrees=True`) or radians (`degrees=False`). For further details, have a look at the [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.from_euler.html) of the `from_euler` function. Here is a short example:

In [17]:
# create a coordinate system by a 90° rotation around the x axis and subsequent 45° rotation around the y axis
cs_05 = tf.LocalCoordinateSystem.construct_from_euler(
    sequence="x", angles=90, degrees=True, coordinates=[1, -1, 0]
)
cs_06 = tf.LocalCoordinateSystem.construct_from_euler(
    sequence="xy", angles=[90,45], degrees=True, coordinates=[2, -2, 0]
)

# create 3d plot
fig = plt.figure()
ax = fig.gca(projection="3d")
fig.canvas.layout.height = "500px"
fig.canvas.layout.width = "500px"

vs.plot_coordinate_system(cs_ref, ax, color="r", label="reference system")
vs.plot_coordinate_system(cs_05, ax, color="g", label="system 5")
vs.plot_coordinate_system(cs_06, ax, color="b", label="system 6")
vs.set_axes_equal(ax)
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Coordinate transformations

It is quite common that there exist a chain or tree like dependency between coordinate systems. We might have a moving object with a local coordinate system which describes its position and orientation towards a fixed reference coordinate system. This object can have another object attached to it, with its position and orientation given in relation to its parent objects coordinate system. If we want to know the attached object coordinate system in relation to the reference coordinate system, we have to perform a coordinate transformation. 

To avoid confusions about the reference systems of each coordinate system, we will use the following naming convention for the coordinate systems: `cs_NAME_in_REFERENCE`. This is a coordinate system with name "NAME" and it's reference system has the name "REFERENCE". Only exception to this convention will be the reference coordinate system "cs_ref", which has no reference. 

The `LocalCoordinateSystem` class provides the `+` and `-` operators to change the reference system easily. The `+` operator will transform a coordinate system to the reference coordinate system of its current reference system:

~~~ python
cs_child_in_ref = cs_child_in_parent + cs_parent_in_ref
~~~
As the naming of the variables already implies, the `+` operator should only be used if there exists a **child-parent relation** between the left-hand side and right-hand side system. 
If two coordinate systems share a **common reference system**, the `-` operator transforms one of those systems into the other:

~~~ python
cs_child_in_parent = cs_child_in_ref - cs_parent_in_ref
~~~

It is important to remember that this operation is in general not commutative, since it involves matrix multiplication which is also not commutative. During those operations, the local system that should be transformed into another coordinate system is always located to the left of the `+` or `-` operator. You can also chain multiple transformations, like this:

~~~ python
cs_A_in_C = cs_A_in_B + cs_B_in_ref - cs_C_in_ref
~~~

Pythons operator associativity ([link](https://www.faceprep.in/python/python-operator-precedence-associativity/)) for the `+` and `-` operator ensures, that all operations are performed from left to right. So in the previously shown example, we first calculate an intermediate coordinate system `cs_A_in_ref` (`cs_A_in_B + cs_B_in_ref`) without actually storing it to a variable and subsequently transform it to the reference coordinate system C (`cs_A_in_ref - cs_C_in_ref`). Keep in mind, that the intermediate results and the coordinate system on the right-hand side of the next operator must either have a child-parent relation (`+` operator) or share a common coordinate system (`-` operator), otherwise the transformation chain produces invalid results.

You can think about both operators in context of a tree like graph structure where all dependency chains lead to a common root coordinate system. The `+` operator moves a coordinate system 1 level higher and closer to the root. Since it's way to the root leads over it's parent coordinate system, the parent is the only valid system than can be used on the right-hand side of the `+` operator. The `-` operator pushes a coorinate system one level lower and further away from the root. It can only be pushed one level deeper, if there is another coordinate system connected to its parent system.

TODO: Add pictures

# Interactive examples

The folowing small interactive examples should give you a better understanding of how the `+` and `-` operators work. The examples provide several sliders to modify the orientations and positions of 2 coordinate systems. From those a third coordinate system is calculated using the `+` and `-` operator. Subsequently, the coordinate systems are plotted in relation to each other. The relevant lines of code, which generate the coordinate systems are:

In [23]:
def coordinate_system_addition(parent_orientation, parent_coordinates, child_orientation, child_coordinates):
    cs_parent_in_ref = tf.LocalCoordinateSystem(orientation=parent_orientation, coordinates=parent_coordinates)
    cs_child_in_parent = tf.LocalCoordinateSystem(orientation=child_orientation, coordinates=child_coordinates)

    cs_child_in_ref = cs_child_in_parent + cs_parent_in_ref

    return [cs_parent_in_ref, cs_child_in_parent, cs_child_in_ref]


def coordinate_system_subtraction(
    sys1_in_ref_orientation, sys1_in_ref_coordinates, sys2_in_ref_orientation, sys2_in_ref_coordinates
):
    cs_sys1_in_ref = tf.LocalCoordinateSystem(orientation=sys1_in_ref_orientation, coordinates=sys1_in_ref_coordinates)
    cs_sys2_in_ref = tf.LocalCoordinateSystem(orientation=sys2_in_ref_orientation, coordinates=sys2_in_ref_coordinates)

    cs_sys2_in_sys1 = cs_sys2_in_ref - cs_sys1_in_ref
    cs_sys1_in_sys2 = cs_sys1_in_ref - cs_sys2_in_ref

    return [cs_sys1_in_ref, cs_sys2_in_ref, cs_sys1_in_sys2, cs_sys2_in_sys1]

Now just execute the following code cells. You don't need to understand them, since they just create the sliders and plots:

In [24]:
cs_ref = tf.LocalCoordinateSystem()


def create_output_widget(window_size=900):
    # create output widget that will hold the figure
    out = widgets.Output(layout={"border": "2px solid black"})

    # create figure inside output widget
    with out:
        fig = plt.figure()
        fig.canvas.layout.height = str(window_size) + "px"
        fig.canvas.layout.width = str(window_size) + "px"
        gs = fig.add_gridspec(3, 2)
        ax_0 = fig.add_subplot(gs[0, 0], projection="3d")
        ax_1 = fig.add_subplot(gs[0, 1], projection="3d")
        ax_2 = fig.add_subplot(gs[1:, 0:], projection="3d")
    return [out, fig, ax_0, ax_1, ax_2]


def setup_axes(axes, limit, title=""):
    axes.set_xlim([-limit, limit])
    axes.set_ylim([-limit, limit])
    axes.set_zlim([-limit, limit])
    axes.set_xlabel("x")
    axes.set_ylabel("y")
    axes.set_zlabel("z")
    axes.set_title(title)
    axes.legend(loc="lower left")


def get_orientation_and_location(t_x, t_y, t_z, r_x, r_y, r_z):
    print("yay")
    rot_angles = np.array([r_x, r_y, r_z], float) / 180 * np.pi

    rot_x = tf.rotation_matrix_x(rot_angles[0])
    rot_y = tf.rotation_matrix_y(rot_angles[1])
    rot_z = tf.rotation_matrix_z(rot_angles[2])

    orientation = np.matmul(rot_z, np.matmul(rot_y, rot_x))
    location = [t_x, t_y, t_z]
    return [orientation, location]


def create_slider(limit, step, label):
    layout = widgets.Layout(width="200px", height="40px")
    style = {"description_width": "initial"}
    return FloatSlider(
        min=-limit, max=limit, step=step, description=label, continuous_update=True, layout=layout, style=style
    )


def create_interactive_plot(function, limit_loc=3, name_sys1="system 1", name_sys2="system 2"):
    step_loc = 0.25

    w_s1_l = dict(
        s1_x=create_slider(limit_loc, step_loc, "x"),
        s1_y=create_slider(limit_loc, step_loc, "y"),
        s1_z=create_slider(limit_loc, step_loc, "z"),
    )

    w_s1_r = dict(
        s1_rx=create_slider(180, 10, "x"), s1_ry=create_slider(180, 10, "y"), s1_rz=create_slider(180, 10, "z")
    )

    w_s2_l = dict(
        s2_x=create_slider(limit_loc, step_loc, "x"),
        s2_y=create_slider(limit_loc, step_loc, "y"),
        s2_z=create_slider(limit_loc, step_loc, "z"),
    )

    w_s2_r = dict(
        s2_rx=create_slider(180, 10, "x"), s2_ry=create_slider(180, 10, "y"), s2_rz=create_slider(180, 10, "z")
    )

    w = {**w_s1_l, **w_s1_r, **w_s2_l, **w_s2_r}

    output = interactive_output(function, w)
    box_0 = VBox([widgets.Label(name_sys1 + " coordinates"), *w_s1_l.values()])
    box_1 = VBox([widgets.Label(name_sys1 + " rotation (deg)"), *w_s1_r.values()])
    box_2 = VBox([widgets.Label(name_sys2 + " coordinates"), *w_s2_l.values()])
    box_3 = VBox([widgets.Label(name_sys2 + " rotation (deg)"), *w_s2_r.values()])
    box = HBox([box_0, box_1, box_2, box_3])
    display(box)

In [26]:
axes_lim = 3
window_size = 1000

[out, fig, ax_0, ax_1, ax_2] = create_output_widget(window_size)


def update_output(s1_x, s1_y, s1_z, s1_rx, s1_ry, s1_rz, s2_x, s2_y, s2_z, s2_rx, s2_ry, s2_rz):

    [parent_orientation, parent_coordinates] = get_orientation_and_location(s1_x, s1_y, s1_z, s1_rx, s1_ry, s1_rz)
    [child_orientation, child_coordinates] = get_orientation_and_location(s2_x, s2_y, s2_z, s2_rx, s2_ry, s2_rz)

    [cs_parent, cs_child, cs_child_ref] = coordinate_system_addition(
        parent_orientation, parent_coordinates, child_orientation, child_coordinates
    )

    coordinates_cr = cs_child_ref.coordinates
    cr_x = coordinates_cr[0]
    cr_y = coordinates_cr[1]
    cr_z = coordinates_cr[2]

    ax_0.clear()
    vs.plot_coordinate_system(cs_ref, ax_0, color="r", label="reference")
    vs.plot_coordinate_system(cs_parent, ax_0, color="g", label="parent")
    ax_0.plot([0, s1_x], [0, s1_y], [0, s1_z], "c--", label="ref -> parent")
    setup_axes(ax_0, axes_lim, "'parent' in reference coordinate system")

    ax_1.clear()
    vs.plot_coordinate_system(cs_ref, ax_1, color="g", label="parent")
    vs.plot_coordinate_system(cs_child, ax_1, color="y", label="child")
    ax_1.plot([0, s2_x], [0, s2_y], [0, s2_z], "m--", label="parent -> child")
    setup_axes(ax_1, axes_lim, "'child' in 'parent' coordinate system")

    ax_2.clear()
    vs.plot_coordinate_system(cs_ref, ax_2, color="r", label="reference")
    vs.plot_coordinate_system(cs_parent, ax_2, color="g", label="parent")
    vs.plot_coordinate_system(cs_child_ref, ax_2, color="y", label="parent + child")
    ax_2.plot([0, s1_x], [0, s1_y], [0, s1_z], "c--", label="ref -> parent")
    ax_2.plot([s1_x, cr_x], [s1_y, cr_y], [s1_z, cr_z], "m--", label="parent -> child")
    setup_axes(ax_2, axes_lim * 2, "'parent' and 'child' in reference coordinate system")


create_interactive_plot(update_output, limit_loc=axes_lim, name_sys1="parent", name_sys2="child")
out

yay
yay


Output(layout=Layout(border='2px solid black'))

In [32]:
axes_lim = 1.5
window_size = 1000

[out_2, fig2, ax2_0, ax2_1, ax2_2] = create_output_widget(window_size)


def update_output2(s1_x, s1_y, s1_z, s1_rx, s1_ry, s1_rz, s2_x, s2_y, s2_z, s2_rx, s2_ry, s2_rz):

    [sys1_orientation, sys1_coordinates] = get_orientation_and_location(s1_x, s1_y, s1_z, s1_rx, s1_ry, s1_rz)
    [sys2_orientation, sys2_coordinates] = get_orientation_and_location(s2_x, s2_y, s2_z, s2_rx, s2_ry, s2_rz)

    [cs_sys1_in_ref, cs_sys2_in_ref, cs_sys1_in_sys2, cs_sys2_in_sys1] = coordinate_system_subtraction(
        sys1_orientation, sys1_coordinates, sys2_orientation, sys2_coordinates
    )
    sys12_o = cs_sys1_in_sys2.coordinates
    sys12_x = sys12_o[0]
    sys12_y = sys12_o[1]
    sys12_z = sys12_o[2]

    sys21_o = cs_sys2_in_sys1.coordinates
    sys21_x = sys21_o[0]
    sys21_y = sys21_o[1]
    sys21_z = sys21_o[2]

    ax2_1.clear()
    vs.plot_coordinate_system(cs_ref, ax2_1, color="g", label="system 1 (reference)")
    vs.plot_coordinate_system(cs_sys2_in_sys1, ax2_1, color="b", label="system 2 - system 1")
    ax2_1.plot([0, sys21_x], [0, sys21_y], [0, sys21_z], "y--", label="system 1 -> system 2")
    setup_axes(ax2_1, axes_lim * 2, "'system 2' in 'system 1'")

    ax2_0.clear()
    vs.plot_coordinate_system(cs_ref, ax2_0, color="b", label="system 2 (reference)")
    vs.plot_coordinate_system(cs_sys1_in_sys2, ax2_0, color="g", label="system_1 - system 2")
    ax2_0.plot([0, sys12_x], [0, sys12_y], [0, sys12_z], "y--", label="system 1 -> system 2")
    setup_axes(ax2_0, axes_lim * 2, "'system 1' in 'system 2'")

    ax2_2.clear()
    vs.plot_coordinate_system(cs_ref, ax2_2, color="r", label="reference")
    vs.plot_coordinate_system(cs_sys1_in_ref, ax2_2, color="g", label="system 1")
    vs.plot_coordinate_system(cs_sys2_in_ref, ax2_2, color="b", label="system 2")
    ax2_2.plot([0, s1_x], [0, s1_y], [0, s1_z], "g--", label="ref -> system 1")
    ax2_2.plot([0, s2_x], [0, s2_y], [0, s2_z], "b--", label="ref -> system 2")
    ax2_2.plot([s1_x, s2_x], [s1_y, s2_y], [s1_z, s2_z], "y--", label="system 1 <-> system 2")
    setup_axes(ax2_2, axes_lim, "'system 1' and 'system 2' in reference coordinate system")


create_interactive_plot(update_output2, limit_loc=axes_lim)
out_2

Output(layout=Layout(border='2px solid black'))